In [17]:
import sys
import os
from PyQt5.QtWidgets import QMainWindow, QApplication, QWidget, QVBoxLayout, QHBoxLayout, QTabWidget, QPushButton, QTextBrowser
import pyqtgraph as pg
import numpy as np
import uproot

In [18]:
class DataReader:
    def __init__(self, filenames,grab):
        self.filenames = filenames
        self.current_index = 0

        self.grab = grab

    def read_data(self):
        filename = self.filenames[self.current_index]
        targettree = uproot.open(filename)['QA_ana']

        if self.grab == "VERTEX":
            gvx = targettree['gvx'].arrays(library='np')['gvx']
            gvy = targettree['gvy'].arrays(library='np')['gvy']
            gvz = targettree['gvz'].arrays(library='np')['gvz']

            vtx0 = [sublist[0] for sublist in gvx]
            vtx1 = [sublist[1] for sublist in gvx]
            vty0 = [sublist[0] for sublist in gvy]
            vty1 = [sublist[1] for sublist in gvy]
            vtz0 = [sublist[0] for sublist in gvz]
            vtz1 = [sublist[1] for sublist in gvz]

            vtx_data = np.array([vtx0, vtx1, vty0, vty1, vtz0, vtz1]).T
            
            return vtx_data
        elif self.grab == "HIT":

            elementid=targettree["elementID"].arrays(library="np")["elementID"]
            detectorid=targettree["detectorID"].arrays(library="np")["detectorID"]

            return np.array([elementid, detectorid])
        else:
            print("DATA NOT SENT!")
    
    def next_file(self):
        self.current_index = (self.current_index + 1) % len(self.filenames)

In [20]:
class ArrayExtractor(QWidget):
    def __init__(self):
        super().__init__()
        self.layout=QVBoxLayout()
        self.vtxBox = QTextBrowser()
        self.hitBox = QTextBrowser()
        self.updateButton=QPushButton("Update")
        self.layout.addWidget(self.vtxBox)
        self.layout.addWidget(self.hitBox)
        self.layout.addWidget(self.updateButton)
        self.setLayout(self.layout)
        
        filenames = sorted([filename for filename in os.listdir("Data") if filename.endswith(".root")])
        self.vtx_reader = DataReader([os.path.join("Data", filename) for filename in filenames],"VERTEX")
        self.hit_reader = DataReader([os.path.join("Data", filename) for filename in filenames],"HIT")
        self.vtx_data = self.vtx_reader.read_data().flatten()
        #self.hit_data = self.hit_reader.read_data().flatten()
        self.vtxBox.setText(np.array2string(self.vtx_data,formatter={'float_kind': lambda x: "%.2f" % x}))
        #self.hitBox.setText(np.array2string(self.hit_data,formatter={'float_kind': lambda x: "%.2f" % x}))
        
        self.updateButton.clicked.connect(self.update_data)
        
    def update_data(self):
        if self.vtx_reader and self.hit_reader:
            self.vtx_reader.next_file()
            self.hit_reader.next_file()
            self.vtx_data = self.vtx_reader.read_data().flatten()
            #self.hit_data = self.hit_reader.read_data().flatten()
            self.vtxBox.setText(np.array2string(self.vtx_reader.read_data().flatten(),formatter={'float_kind': lambda x: "%.2f" % x}))
            #self.hitBox.setText(np.array2string(self.hit_reader.read_data().flatten(),formatter={'float_kind': lambda x: "%.2f" % x}))
            self.layout.replaceWidget(self.layout.itemAt(0),self.vtxBox)
            #self.layout.addWidget(self.hitBox)
            self.layout.addWidget(self.updateButton)
            self.setLayout(self.layout)
                
        
        